In [1]:
!pip install kaggle
import os
# Replace with your Kaggle API credentials
os.environ['KAGGLE_USERNAME'] = "YOUR_KAGGLE_USERNAME"
os.environ['KAGGLE_KEY'] = "YOUR_KAGGLE_KEY"

!kaggle datasets download nih-chest-xrays/data
!unzip data.zip


Streaming output truncated to the last 5000 lines.
  inflating: images_012/images/00028897_016.png  
  inflating: images_012/images/00028897_017.png  
  inflating: images_012/images/00028897_018.png  
  inflating: images_012/images/00028897_019.png  
  inflating: images_012/images/00028897_020.png  
  inflating: images_012/images/00028898_000.png  
  inflating: images_012/images/00028899_000.png  
  inflating: images_012/images/00028899_001.png  
  inflating: images_012/images/00028899_002.png  
  inflating: images_012/images/00028900_000.png  
  inflating: images_012/images/00028901_000.png  
  inflating: images_012/images/00028901_001.png  
  inflating: images_012/images/00028902_000.png  
  inflating: images_012/images/00028902_001.png  
  inflating: images_012/images/00028902_002.png  
  inflating: images_012/images/00028902_003.png  
  inflating: images_012/images/00028902_004.png  
  inflating: images_012/images/00028902_005.png  
  inflating: images_012/images/00028902_006.png  

In [2]:
import numpy as np
import pandas as pd
import os
from glob import glob
from itertools import chain
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split

In [3]:
#Importing data
df = pd.read_csv("Data_Entry_2017.csv")
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112120 entries, 0 to 112119
Data columns (total 12 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Image Index                  112120 non-null  object 
 1   Finding Labels               112120 non-null  object 
 2   Follow-up #                  112120 non-null  int64  
 3   Patient ID                   112120 non-null  int64  
 4   Patient Age                  112120 non-null  int64  
 5   Patient Gender               112120 non-null  object 
 6   View Position                112120 non-null  object 
 7   OriginalImage[Width          112120 non-null  int64  
 8   Height]                      112120 non-null  int64  
 9   OriginalImagePixelSpacing[x  112120 non-null  float64
 10  y]                           112120 non-null  float64
 11  Unnamed: 11                  0 non-null       float64
dtypes: float64(3), int64(5), object(4)
memory usage: 10.3+ MB


In [5]:
df.rename(columns={"Finding Labels":"Xray_labels","Image Index":"Image_file"},inplace=True)

In [6]:
image_paths = {os.path.basename(x): x for x in glob(os.path.join('..', 'images*', '*', '*.png'))}
#creating new column: file_path
df["File_path"] = df["Image_file"].map(image_paths)

In [7]:
df.sample(3)

,Image_file,Xray_labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,File_path
12779,00003356_000.png,No Finding,0,3356,71,M,AP,2500,2048,0.171,0.171,NaN,NaN
37122,00009798_006.png,No Finding,6,9798,34,M,PA,2992,2991,0.143,0.143,NaN,NaN
39805,00010394_002.png,No Finding,2,10394,36,M,PA,2992,2991,0.143,0.143,NaN,NaN


In [8]:
#Xray_labels column
df['Xray_labels'] = df['Xray_labels'].map(lambda x: x.replace('No Finding', ''))
xray_labels = np.unique(list(chain(*df['Xray_labels'].map(lambda x: x.split('|')).tolist())))
print("All diseases: ",xray_labels)
xray_labels = [x for x in xray_labels if len(x)>0]

for c_label in xray_labels:
    if len(c_label)>1:
        df[c_label] = df['Xray_labels'].map(lambda finding: 1 if c_label in finding else 0)

df.sample(5)

All diseases:  ['' 'Atelectasis' 'Cardiomegaly' 'Consolidation' 'Edema' 'Effusion'
 'Emphysema' 'Fibrosis' 'Hernia' 'Infiltration' 'Mass' 'Nodule'
 'Pleural_Thickening' 'Pneumonia' 'Pneumothorax']


,Image_file,Xray_labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,...,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax
94754,00024486_000.png,,0,24486,23,F,PA,2048,2500,0.168000,...,0,0,0,0,0,0,0,0,0,0
4117,00001112_000.png,,0,1112,64,F,PA,2021,2021,0.194311,...,0,0,0,0,0,0,0,0,0,0
66774,00016508_015.png,Infiltration,15,16508,51,F,AP,2500,2048,0.168000,...,0,0,0,0,1,0,0,0,0,0
92121,00022993_028.png,Atelectasis|Consolidation|Infiltration|Nodule,28,22993,58,M,AP,3056,2544,0.139000,...,0,0,0,0,1,0,1,0,0,0
51558,00013026_001.png,,1,13026,59,M,PA,2992,2991,0.143000,...,0,0,0,0,0,0,0,0,0,0


In [ ]:

selected_columns = ['Image_file', 'Patient ID'] + xray_labels
df_selected = df[selected_columns]
df_selected.sample(5)


,Image_file,Patient ID,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax
47403,00012045_035.png,12045,0,0,0,0,0,0,0,0,0,0,0,0,0,0
110458,00030079_050.png,30079,0,0,0,0,0,0,0,0,1,0,1,0,0,0
26206,00006875_025.png,6875,0,0,0,0,0,0,0,0,0,0,0,0,0,0
84966,00020945_017.png,20945,0,0,0,0,0,0,0,0,0,0,1,0,0,0
85292,00021024_012.png,21024,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [11]:
df_selected.to_csv('df_selected.csv', index=False)


In [12]:
!pip install pillow
from PIL import Image
import os
from tqdm import tqdm

image_dir_paths = [f"images_00{i}/images" if i < 10 else f"images_0{i}/images" for i in range(1, 13)]

target_size = (320, 320)

for image_dir in image_dir_paths:
    if not os.path.exists(image_dir):
        print(f"Warning: Directory {image_dir} not found.")
        continue
    image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

    for image_file in tqdm(image_files, desc=f"Processing images in {image_dir}"):
        try:
            image_path = os.path.join(image_dir, image_file)
            img = Image.open(image_path)
            img = img.resize(target_size, Image.Resampling.LANCZOS)
            img.save(image_path)
        except Exception as e:
            print(f"Error processing {image_file}: {e}")


Processing images in images_012/images: 100%|██████████| 7121/7121 [04:05<00:00, 28.98it/s]


In [ ]:
import os
import shutil
from tqdm import tqdm

# Create the target directory if it doesn't exist
target_dir = "images"
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# Define the source directories
image_dir_paths = [f"images_00{i}/images" if i < 10 else f"images_0{i}/images" for i in range(1, 13)]

# Move the images to the 'images' directory and remove the original folders
for image_dir in image_dir_paths:
    if os.path.exists(image_dir):
        image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

        for image_file in tqdm(image_files, desc=f"Moving images from {image_dir}"):
            source_path = os.path.join(image_dir, image_file)
            target_path = os.path.join(target_dir, image_file)
            shutil.move(source_path, target_path)

        # Remove the source directory after moving the files
        shutil.rmtree(os.path.dirname(image_dir))
    else:
        print(f"Warning: Directory {image_dir} not found.")

Moving images from images_012/images: 100%|██████████| 7121/7121 [00:00<00:00, 14611.61it/s]


In [ ]:

import os
import shutil

# Define the source directories and files
source_dirs = "images"
source_files = [
    "ARXIV_V5_CHESTXRAY.pdf",
    "BBox_List_2017.csv",
    "Data_Entry_2017.csv",
    "FAQ_CHESTXRAY.pdf",
    "LOG_CHESTXRAY.pdf",
    "README_CHESTXRAY.pdf",
    "test_list.txt",
    "train_val_list.txt",
    "df_selected.csv"
]

# Create a temporary directory to store files to be zipped
!mkdir temp_zip_dir

# Copy directories to the temp directory
if os.path.exists(source_dirs):
    shutil.copytree(source_dirs, os.path.join("temp_zip_dir", source_dirs))
else:
  print(f"Directory not found: {source_dirs}")

# Copy files to the temp directory
for file_name in source_files:
    if os.path.exists(file_name):
        shutil.copy(file_name, "temp_zip_dir")
    else:
        print(f"File not found: {file_name}")


# Create the zip archive
shutil.make_archive("E_compressed", 'zip', "temp_zip_dir")

#remove temp dir
!rm -rf temp_zip_dir

print("Archive E_compressed.zip has been created")

Archive E_compressed.zip has been created


In [16]:
import os

# Get the size of the zip file in bytes
file_size_bytes = os.path.getsize("E_compressed.zip")

# Convert the size to gigabytes
file_size_gb = file_size_bytes / (1024 ** 3)

print(f"The size of E_compressed.zip is: {file_size_gb:.2f} GB")

# Download the file to your local machine using the files tab in the left sidebar of Google Colab.
# Alternatively, you can mount your Google Drive and copy the file there:

from google.colab import drive
drive.mount('/content/drive')

# Copy the zip file to your Google Drive
!cp E_compressed.zip "/content/drive/My Drive/E_compressed.zip"  # Change the destination path if needed

print("File copied to your Google Drive: /content/drive/My Drive/E_compressed.zip")


The size of E_compressed.zip is: 4.66 GB
Mounted at /content/drive
File copied to your Google Drive: /content/drive/My Drive/E_compressed.zip
